In [1]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler(num_train_timesteps=1000, beta_start=0.0001, beta_end=0.02)
print("timesteps max and min:", scheduler.timesteps.max(), scheduler.timesteps.min())
print(scheduler.timesteps[:10])


timesteps max and min: tensor(999) tensor(0)
tensor([999, 998, 997, 996, 995, 994, 993, 992, 991, 990])


In [2]:
import sys
import torch
sys.path.append("../../../")

from train5 import denoising_step, forward_diffusion

num_timesteps = 1000
betas = torch.linspace(1e-4, 0.02, num_timesteps)
alphas = 1 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
noise_schedule = {
    "betas": betas,
    "alphas": alphas,
    "alphas_cumprod": alphas_cumprod,
}
print("betas:", betas[:10])
print("alphas:", alphas[:10])
print("alphas_cumprod:", alphas_cumprod[:10])


2024-10-05 20:40:46.300366: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-05 20:40:46.343350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-10-05 20:40:48,516 - datasets - INFO - PyTorch version 2.0.1 available. (config.py:54)
2024-10-05 20:40:48,517 - datasets - INFO - Tensor

betas: tensor([1.0000e-04, 1.1992e-04, 1.3984e-04, 1.5976e-04, 1.7968e-04, 1.9960e-04,
        2.1952e-04, 2.3944e-04, 2.5936e-04, 2.7928e-04])
alphas: tensor([0.9999, 0.9999, 0.9999, 0.9998, 0.9998, 0.9998, 0.9998, 0.9998, 0.9997,
        0.9997])
alphas_cumprod: tensor([0.9999, 0.9998, 0.9996, 0.9995, 0.9993, 0.9991, 0.9989, 0.9986, 0.9984,
        0.9981])


In [3]:
# create a dummy denoising model
class DummyDenoisingModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Linear(10, 10)
    
    def forward(self, x, t):
        return self.net(x)


torch.manual_seed(10)
denoising_model = DummyDenoisingModel()

In [4]:
torch.manual_seed(1)
X = torch.randn(1, 10)
noise = torch.randn_like(X)
X.numpy()[0], noise.numpy()[0]

(array([ 0.66135216,  0.2669241 ,  0.06167726,  0.6213173 , -0.45190597,
        -0.16613023, -1.5227685 ,  0.38168392, -1.0276086 , -0.5630528 ],
       dtype=float32),
 array([-0.89229053, -0.05825018, -0.19550958, -0.96563596,  0.42241532,
         0.267317  , -0.42119515, -0.5107    , -1.5726652 , -0.12324776],
       dtype=float32))

In [5]:
torch.manual_seed(0)
t = 0

with torch.no_grad():
    sample = denoising_step(
        denoising_model=denoising_model,
        x_t=X,
        t=t,
        noise_schedule=noise_schedule,
    )
    print("sample:", sample.numpy()[0, 0, 0])


x_t mean: -0.1738511174917221
t: 0
model_output mean: 0.1872388869524002
pred_original_sample mean (before clipping): -0.17573246359825134
alpha_prod_t 0.9998999834060669
alpha_prod_t_prev 1.0
beta_prod_t 0.00010001659393310547
beta_prod_t_prev 0.0
current_alpha_t 0.9998999834060669
current_beta_t 0.00010001659393310547
pred_original_sample mean -0.12070973962545395
pred_original_sample_coeff 1.0
current_sample_coeff 0.0
sample: [ 0.66127455  0.2645179   0.05515494  0.62613356 -0.45403624 -0.16315319
 -1.          0.36983928 -1.         -0.56682825]


In [6]:
# Would diffuser scheduler's step function be the same as the one we implemented?
torch.manual_seed(0)
model_output = denoising_model(X, torch.tensor(t))
with torch.no_grad():
    step_output = scheduler.step(model_output=model_output, timestep=t, sample=X)
    print("sample:", step_output.prev_sample.numpy()[0])

x_t mean: -0.1738511174917221
t: 0
model_output mean: 0.1872388869524002
pred_original_sample mean (before clipping): -0.17573246359825134
thresholding: False
clip_sample: True
alpha_prod_t 0.9998999834060669
alpha_prod_t_prev 1.0
beta_prod_t 0.00010001659393310547
beta_prod_t_prev 0.0
current_alpha_t 0.9998999834060669
current_beta_t 0.00010001659393310547
pred_original_sample mean: -0.12070973962545395
pred_prev_sample mean: -0.12070973962545395
current_sample_coeff: 0.0
sample: [ 0.66127455  0.2645179   0.05515494  0.62613356 -0.45403624 -0.16315319
 -1.          0.36983928 -1.         -0.56682825]


## Also check if DDPMScheduler's add_noise function is the same as forward_diffusion function

In [7]:
noised = scheduler.add_noise(original_samples=X, noise=noise, timesteps=torch.tensor(1))
noised


tensor([[ 0.6480,  0.2660,  0.0588,  0.6069, -0.4456, -0.1621, -1.5288,  0.3741,
         -1.0508, -0.5648]])

In [8]:
forward_diffusion(x_0=X, t=torch.tensor(1), noise_schedule=noise_schedule, noise=noise)

(tensor([[[[ 0.6480,  0.2660,  0.0588,  0.6069, -0.4456, -0.1621, -1.5288,
             0.3741, -1.0508, -0.5648]]]]),
 tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224,  0.2673, -0.4212, -0.5107,
          -1.5727, -0.1232]]))